# Stackelberg Best Response
## (in a 2 player, 2 targets SG)

This notebook finds with linear programming the maxmin solution to a 2 players-2 targets SG, which correspond to the best response to a Stackelberg adversary. Analitical solution to this problem can be found here: https://www.overleaf.com/read/xmcpdzwsmrty

According to that calculations, calling $V_m$ and $V_M$ the minimum and the maximum targets values respectively, the defender best strategy is $(\frac{V_m}{V_m+V_M},\frac{V_M}{V_m+V_M})$ and the relative payoff is: $\frac{-V_MV_m}{V_m+V_M}$

In [3]:
from gurobipy import *

The approach used here is taken from "Introduction to Operative Research" by Hillier & Lieberman (section 14.5)

In [2]:
try:

    # Create a new model
    m = Model("SSG")

    # Create variables
    x1 = m.addVar(vtype=GRB.CONTINUOUS, name="x1")
    x2 = m.addVar(vtype=GRB.CONTINUOUS, name="x2")
    x3 = m.addVar(lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x3")
    v_m = 5
    v_M = 6
    # Set objective
    m.setObjective(x3, GRB.MAXIMIZE)

    # Add constraint:
    m.addConstr(x1 + x2 == 1, "c0")
    m.addConstr(-v_m * x2 >= x3, "c1")
    m.addConstr(-v_M * x1 >= x3, "c2")

    m.optimize()

    for v in m.getVars():
        print(v.varName, v.x)

    print('Obj:', m.objVal)

except GurobiError:
    print('Error reported')

Optimize a model with 3 rows, 3 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 1 columns
Presolve time: 0.02s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   3.750000e-01   0.000000e+00      0s
       1   -2.7272727e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds
Optimal objective -2.727272727e+00
x1 0.4545454545454546
x2 0.5454545454545454
x3 -2.7272727272727275
Obj: -2.7272727272727275


We can compare these results with the analytical ones:

In [3]:
analitycal_obj = -(v_M * v_m)/(v_m + v_M)
analitycal_x1 = v_m / (v_m + v_M)
analitycal_x2 = v_M / (v_m + v_M)
print(analitycal_x1 ,analitycal_x2 , analitycal_obj)

0.45454545454545453 0.5454545454545454 -2.727272727272727


In [35]:
values = (1,3,1,2,1,3)

In [36]:
m = Model("SSG")
targets = list(range(len(values)))
strategy = []
for t in targets:
    strategy.append(m.addVar(vtype=GRB.CONTINUOUS, name="x"+str(t)))
v = m.addVar(lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="v")
m.setObjective(v, GRB.MAXIMIZE)
for t in targets:
    terms = [-values[t]*strategy[i] for i in targets if i != t]
    m.addConstr(sum(terms) - v >= 0, "c"+str(t))
m.addConstr(sum(strategy) == 1, "c"+str(len(targets)))
m.params.outputflag = 0
m.optimize()
for v in m.getVars():
    print(v.varName, v.x)

print('Obj:', m.objVal)
print([float(s.x) for s in strategy])

x0 0.0
x1 0.4285714285714286
x2 0.0
x3 0.14285714285714285
x4 0.0
x5 0.4285714285714286
v -1.7142857142857142
Obj: -1.7142857142857142
[0.0, 0.4285714285714286, 0.0, 0.14285714285714285, 0.0, 0.4285714285714286]


In [8]:
[float(s.x) for s in strategy]

[0.0, 0.4285714285714286, 0.0, 0.14285714285714285, 0.0, 0.4285714285714286]

In [30]:
f = 5

In [31]:
terms = [-values[f]*strategy[i].x for i in targets if i != f]

In [32]:
sum(terms)

-1.7142857142857144